<a href="https://colab.research.google.com/github/hirokimituya/stock-price-analysis/blob/main/technical_analysis/%E9%8A%98%E6%9F%84%E9%81%B8%E5%AE%9A%E9%96%A2%E6%95%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Google Colaboratoryに画像を保存できるように設定
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/')

Mounted at /content/drive


In [ ]:
# Ta-Libの代替としてpandas_taを利用している。Ta-Libの関数名を小文字にしたものがあるイメージ
!pip install pandas_ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=e4267784e13a2144d1614e11d047fd00c9dd38e2295c3e20c8d59a1e2957594e
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [ ]:
!pip install yahoo_finance_api2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for yahoo_finance_api2: filename=yahoo_finance_api2-0.0.12-py3-none-any.whl size=3714 sha256=f596c23e66dc407ba29c1d7cfb5274ea95ea48bcdad874b6cfb986cb329e670e
  Stored in directory: /root/.cache/pip/wheels/e2/40/e3/f3d8054a41d7d85065555037525f48f56119cafbc5fcddf427
Successfully built yahoo_finance_api2


# 銘柄コードから株価情報を取得する関数

In [ ]:
from yahoo_finance_api2 import share as yapi2
from yahoo_finance_api2.exceptions import YahooFinanceError
import datetime as dt
from datetime import date,timedelta
import pandas as pd

# 株価データを取得するメソッド
def get_stock_data_yapi2(code, period_type='year', period=10):
    """株価データを取得する

    :param code: 取得する株価データの銘柄コード
    :param period_type: 取得する株価データの期間の単位（例: 'year', 'month', 'week', 'day'）
    :param period: 取得する株価データの期間の数値
    :return: 株価データのデータフレーム（もし株価データが存在しない場合は、None）
    """
    if period_type == 'month':
        period_type_value = yapi2.PERIOD_TYPE_MONTH
    elif period_type == 'week':
        period_type_value = yapi2.PERIOD_TYPE_WEEK
    elif period_type == 'day':
        period_type_value = yapi2.PERIOD_TYPE_DAY
    else:
        period_type_value = yapi2.PERIOD_TYPE_YEAR

    try:
        data = yapi2.Share(f'{code}.T').get_historical(
            period_type_value,
            period,
            yapi2.FREQUENCY_TYPE_DAY,
            1
        )
    except YahooFinanceError as e:
        print(e.message)
        print(f'銘柄コード[{code}]の株価情報を取得できなかったため、スキップしました。')
        return None

    df = pd.DataFrame(data)

    try:
        # タイムスタンプをDateTime型に変更
        df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
    except KeyError as e:
        print(e)
        print(f'銘柄コード[{code}]の株価情報を取得できなかったため、スキップしました。')
        return None

    # 日本時間へ変換
    df['Date'] = df['datetime'] + dt.timedelta(hours=9)

    # インデックスをDateカラムに設定
    df.index = pd.to_datetime(df['Date'].dt.date)

    # 不要なカラムを削除
    del df['Date'], df['datetime'], df['timestamp']

    # 各カラムの最初の文字を大文字に変換
    for col in df.columns:
        df[f'{col.capitalize()}'] = df[col]
        del df[col]

    return df

In [ ]:
get_stock_data_yapi2(9501)

,Open,High,Low,Close,Volume
Date,,,,,
2013-05-21,810.0,841.0,770.0,815.0,549781900
2013-05-22,770.0,794.0,730.0,737.0,293562700
2013-05-23,710.0,775.0,615.0,639.0,374059400
2013-05-24,689.0,723.0,643.0,702.0,639591296
2013-05-27,671.0,703.0,651.0,651.0,246788700
...,...,...,...,...,...
2023-05-12,487.0,504.0,484.0,499.0,25367600
2023-05-15,504.0,514.0,498.0,510.0,22438100
2023-05-16,511.0,530.0,501.0,522.0,59033600


# JPX400の銘柄一覧を取得する関数

In [ ]:
from pandas.io.formats.info import DataFrameTableBuilderNonVerbose
import requests
import pandas as pd

def get_ticker_jpx400():
    """JPX400の銘柄コード一覧を取得する

    :return: JPX400の銘柄コード一覧
    """

    url = "https://indexes.nikkei.co.jp/nkave/archives/file/jpx_nikkei_index_400_weight_jp.csv"
    r = requests.get(url)
    with open('jpx_nikkei_index_400_weight_jp.csv', 'wb') as output:
        output.write(r.content)

    df = pd.read_csv("./jpx_nikkei_index_400_weight_jp.csv", encoding='shift_jis')
    # 不要なレコードとカラムを削除
    df = df.iloc[:-1]   # 最後の行は説明文のため除外
    df = df[['銘柄名', 'コード', '業種']]   # 必要なカラムのみにする
    df['コード'] = df['コード'].astype(int) # コードを整数に変更

    return df

In [ ]:
# JPX400の銘柄コードを取得
jpx400 = get_ticker_jpx400()
jpx400

,銘柄名,コード,業種
0,ニッスイ,1332,水産・農林業
1,ウエストホールディングス,1407,建設業
2,ショーボンドホールディングス,1414,建設業
3,ミライト・ワン,1417,建設業
4,ＩＮＰＥＸ,1605,鉱業
...,...,...,...
394,ニトリホールディングス,9843,小売業
395,ミスミグループ本社,9962,卸売業
396,ファーストリテイリング,9983,小売業
397,ソフトバンクグループ,9984,情報・通信業


In [ ]:
# ニッスイの株価情報を取得
get_stock_data_yapi2(jpx400.loc[jpx400['銘柄名'] == 'ニッスイ', 'コード'].values[0])

,Open,High,Low,Close,Volume
Date,,,,,
2013-05-21,231.0,235.0,227.0,232.0,3023300
2013-05-22,235.0,238.0,232.0,234.0,3742100
2013-05-23,232.0,233.0,210.0,210.0,6456600
2013-05-24,216.0,224.0,204.0,217.0,5339300
2013-05-27,210.0,213.0,205.0,206.0,2735200
...,...,...,...,...,...
2023-05-12,588.0,624.0,584.0,622.0,5378900
2023-05-15,626.0,630.0,616.0,628.0,3150800
2023-05-16,630.0,632.0,625.0,628.0,1763900


# 東証上場の銘柄一覧を取得する関数

In [ ]:
from pandas.io.formats.info import DataFrameTableBuilderNonVerbose
import requests
import pandas as pd

def get_ticker_tse(segment='prime'):
    """東証上場の銘柄コード一覧を取得する

    :param segment: 東証上場の市場区分（例: 'prime' or 'standard' or 'grows'）
    :return: 東証上場の銘柄コード一覧
    """
    segments = {
        'prime': 'プライム',
        'standard': 'スタンダード',
        'grows': 'グロース'
    }

    if segment not in segments:
        raise Exception("get_ticker_tse関数のsegment引数の値は'prime', 'standard', 'grows'のいずれかにしてください")

    url = "https://www.jpx.co.jp/markets/statistics-equities/misc/tvdivq0000001vg2-att/data_j.xls"
    r = requests.get(url)
    with open('data_j.xls', 'wb') as output:
        output.write(r.content)

    df = pd.read_excel("./data_j.xls")

    # 引数の市場区分のみに絞り込み
    df = df[df['市場・商品区分'].str.startswith(segments[segment])]

    # カラムの削除とカラム名変更
    df = df[['銘柄名', 'コード', '33業種区分']]   # 必要なカラムのみにする
    df = df.rename(columns={'33業種区分': '業種'})
    df['コード'] = df['コード'].astype(int) # コードを整数に変更

    return df

In [ ]:
# 東証プライム市場の銘柄コードを取得
tse_prime = get_ticker_tse('prime')
tse_prime

,銘柄名,コード,業種
0,極洋,1301,水産・農林業
16,ニッスイ,1332,水産・農林業
17,マルハニチロ,1333,水産・農林業
33,雪国まいたけ,1375,水産・農林業
35,サカタのタネ,1377,水産・農林業
...,...,...,...
4262,サンドラッグ,9989,小売業
4263,サックスバー ホールディングス,9990,小売業
4264,ジェコス,9991,卸売業
4267,グローセル,9995,卸売業


# 株価情報を出来高や株価、パーフェクトオーダーでフィルタリングする関数

In [ ]:
import pandas_ta as ta
import math

def is_perfect_order(stock_df, mas, period_type='D', reverse=False):
    """株価情報がパーフェクトオーダーどうかの判定

    :param stock_df: 対象の株価情報のデータフレーム
    :param mas: 移動平均線に使用するためのリスト（例: [5, 25, 75]）
    :param period_type: 期間をどの単位にするか（'W' or 'M'）
    :param reverse: 下降トレンドのパーフェクトオーダーを検出する場合は、True / 上昇トレンドの場合は、False
    :return: 株価情報がパーフェクトオーダーの場合は、True / それ以外の場合は、False
    """

    df = stock_df.copy()

    # 月足、周足にリサンプリング
    if period_type in ('M', 'W'):
        resampled = df.resample(period_type)
        df = resampled.aggregate({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'})
    
    close = df['Close']
    
    # 移動平均線の算出
    for ma in mas:
        df[f'ma{ma}{period_type}'] = ta.sma(close, ma)
    
    # パーフェクトオーダーかどうかの判定
    conditions = []
    try:
        if not reverse:
            # 上昇トレンド
            for i in range(len(mas) - 1):
                conditions.append(math.ceil(df[f'ma{mas[i]}{period_type}'].values[-1]) >= math.ceil(df[f'ma{mas[i + 1]}{period_type}'].values[-1]))
        else:
            # 下降トレンド
            for i in range(len(mas) - 1):
                conditions.append(math.ceil(df[f'ma{mas[i]}{period_type}'].values[-1]) <= math.ceil(df[f'ma{mas[i + 1]}{period_type}'].values[-1]))
    except TypeError:
        # 株価情報が少なくて移動平均線が取得できない場合は、無条件でFalseでリターンする
        return False

            
    result = all(conditions)

    return result

In [ ]:
def filtered_ticker(df, min_volume=None, max_price=None, perfect_order={}):
    """データフレームの銘柄コードから株価情報を取得して、指定された出来高や株価でフィルタリングする

    :param df: 対象のデータフレーム
    :param min_volume: 出来高の最小値を指定
    :param max_price: 株価の最大値を指定
    :param perfect_order: パーフェクトオーダーの判定に使用する辞書（例: {'mas': [5, 20, 60], 'period_type': 'D', 'reverse': False}）（辞書のキー'mas'のみ必須）
    :return: 出来高や株価でフィルタリングしたデータフレーム
    """
    # 空のDataFrameを作成
    result_df = pd.DataFrame()

    # 各銘柄についてループ
    for index, row in df.iterrows():
        # コードを取得
        code = row['コード']

        # 株価情報を取得
        if not perfect_order:
            stock_data = get_stock_data_yapi2(code, period_type='day', period=2)
        else:
            largest_mas = perfect_order.get('mas')[-1]
            period_type = perfect_order.get('period_type', 'D')
            if period_type == 'D':
                period = math.ceil(largest_mas / 15)
                stock_data = get_stock_data_yapi2(code, period_type='month', period=period)
            elif period_type == 'W':
                period = math.ceil(largest_mas * 7 / 365)
                stock_data = get_stock_data_yapi2(code, period_type='year', period=period)
            elif period_type == 'M':
                period = math.ceil(largest_mas / 10)
                stock_data = get_stock_data_yapi2(code, period_type='year', period=period)
            else:
                raise ValueError('引数perfect_orderは"D"か"W"か"M"のいずれかを指定してください')
        # 株価情報が取得できなかったら、次の銘柄コードに遷移
        if stock_data is None:
            continue

        # 出来高の条件に合わない場合、次の銘柄コードに遷移
        volume = stock_data['Volume'][-1]
        if min_volume is not None and volume < min_volume:
            continue
        
        # 価格の条件に合わない場合、次の銘柄コードに遷移
        price = stock_data['Close'][-1]
        if max_price is not None and price > max_price:
            continue
        
        # パーフェクトオーダーの条件に合わない場合、次の銘柄コードに遷移
        if perfect_order:
            mas = perfect_order.get('mas')
            period_type = perfect_order.get('period_type', 'D')
            reverse = perfect_order.get('reverse', False)
            if not is_perfect_order(stock_data, mas=mas, period_type=period_type, reverse=reverse):
                continue

        # 銘柄名と業種を設定
        filtered_df = pd.DataFrame({
            '銘柄名': [row['銘柄名']],
            'コード': [code],
            '業種': [row['業種']],
            '出来高': [volume],
            '株価': [price],
        }, index=[index])

        # 結果をresult_dfに追加
        result_df = pd.concat([result_df, filtered_df])

    return result_df

## JPX400をフィルタリング


In [ ]:
# JPX400の銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 20万株以上
# 株価 ... 5000円以下
filtered_jpx400 = filtered_ticker(jpx400, min_volume=200000, max_price=5000)
filtered_jpx400

,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,1484800,620.0
1,ウエストホールディングス,1407,建設業,211800,2951.0
3,ミライト・ワン,1417,建設業,208200,1794.0
4,ＩＮＰＥＸ,1605,鉱業,7606500,1493.0
5,安藤・間,1719,建設業,574300,1013.0
...,...,...,...,...,...
382,エヌ・ティ・ティ・データ,9613,情報・通信業,3809500,1949.0
387,ＳＣＳＫ,9719,情報・通信業,458200,2230.0
389,メイテック,9744,サービス業,219300,2332.0
395,ミスミグループ本社,9962,卸売業,684800,3530.0


In [ ]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_jpx400 = filtered_ticker(filtered_jpx400, perfect_order={'mas': [5, 20, 60], 'period_type': 'W'})
filtered_perfect_order_week_jpx400

,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,1484800,620.0
3,ミライト・ワン,1417,建設業,208200,1794.0
5,安藤・間,1719,建設業,574300,1013.0
7,大成建設,1801,建設業,923000,4805.0
8,大林組,1802,建設業,2418000,1142.0
...,...,...,...,...,...
375,北海道電力,9509,電気・ガス業,1085800,539.0
376,電源開発,9513,電気・ガス業,1280500,2120.0
379,東京瓦斯,9531,電気・ガス業,2238000,2940.0
395,ミスミグループ本社,9962,卸売業,684800,3530.0


In [ ]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_jpx400 = filtered_ticker(filtered_perfect_order_week_jpx400,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_jpx400

,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,1484800,620.0
3,ミライト・ワン,1417,建設業,208200,1794.0
5,安藤・間,1719,建設業,574300,1013.0
7,大成建設,1801,建設業,923000,4805.0
8,大林組,1802,建設業,2418000,1142.0
...,...,...,...,...,...
373,関西電力,9503,電気・ガス業,3966200,1521.0
375,北海道電力,9509,電気・ガス業,1085800,539.0
379,東京瓦斯,9531,電気・ガス業,2238000,2940.0
395,ミスミグループ本社,9962,卸売業,684800,3530.0


## 東証プライムをフィルタリング

In [ ]:
# 東証プライムの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 20万株以上
# 株価 ... 5000円以下
filtered_tse_prime = filtered_ticker(tse_prime, min_volume=200000, max_price=5000)
filtered_tse_prime

Not Found: No data found, symbol may be delisted
銘柄コード[25935]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[8096]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
16,ニッスイ,1332,水産・農林業,1484800,620.0
17,マルハニチロ,1333,水産・農林業,218200,2560.0
59,ミライト・ワン,1417,建設業,208200,1794.0
61,タマホーム,1419,建設業,304800,3880.0
69,Ｒｏｂｏｔ Ｈｏｍｅ,1435,建設業,557500,195.0
...,...,...,...,...,...
4195,アークランズ,9842,小売業,217700,1613.0
4206,吉野家ホールディングス,9861,小売業,213700,2559.0
4246,ミスミグループ本社,9962,卸売業,684800,3530.0
4261,スズケン,9987,卸売業,244100,3860.0


In [ ]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_prime = filtered_ticker(filtered_tse_prime, perfect_order={'mas': [5, 20, 60], 'period_type': 'W'})
filtered_perfect_order_week_tse_prime

,銘柄名,コード,業種,出来高,株価
16,ニッスイ,1332,水産・農林業,1484800,620.0
17,マルハニチロ,1333,水産・農林業,218200,2560.0
59,ミライト・ワン,1417,建設業,208200,1794.0
61,タマホーム,1419,建設業,304800,3880.0
69,Ｒｏｂｏｔ Ｈｏｍｅ,1435,建設業,557500,195.0
...,...,...,...,...,...
4062,東京瓦斯,9531,電気・ガス業,2238000,2940.0
4192,オートバックスセブン,9832,卸売業,273100,1532.0
4206,吉野家ホールディングス,9861,小売業,213700,2559.0
4246,ミスミグループ本社,9962,卸売業,684800,3530.0


In [ ]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_prime = filtered_ticker(filtered_perfect_order_week_tse_prime,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_prime

,銘柄名,コード,業種,出来高,株価
16,ニッスイ,1332,水産・農林業,1484800,620.0
17,マルハニチロ,1333,水産・農林業,218200,2560.0
59,ミライト・ワン,1417,建設業,208200,1794.0
61,タマホーム,1419,建設業,304800,3880.0
223,安藤・間,1719,建設業,574300,1013.0
...,...,...,...,...,...
4062,東京瓦斯,9531,電気・ガス業,2238000,2940.0
4192,オートバックスセブン,9832,卸売業,273100,1532.0
4206,吉野家ホールディングス,9861,小売業,213700,2559.0
4246,ミスミグループ本社,9962,卸売業,684800,3530.0
